# Tutorial on "Graph Neural Networks from Scratch"

## 1. Loading the graph

We will use ... This is most popular ... It consists of ...

Now we can transform it to more usable format...

## 2. Factorizing the adjacency matrix

Why do we need to factorize the adjacency matrix? Something with message passing, video with description: [https://www.youtube.com/watch?v=ijmxpItkRjc]()

After that step is done, we can move on to model...

## 3. Graph Convolution Layer